In [ ]:
import tensorflow as tf
# import Sequential model
from tensorflow.keras.models import Sequential
# import typical layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
# interface with the TensorBoard
from tensorflow.keras.callbacks import TensorBoard
import pickle
import os
import time # make this model unique, and not end up overwriting a model

directPath = os.getcwd()


"1. Scale input data"
# Read saved data
X = pickle.load(open(directPath + "\\X.pickle", "rb"))
y = pickle.load(open(directPath + "\\y.pickle", "rb"))

# Scale data/image by divide it by 255
X = X/255.0


"2. Set up CNN model"
# Set name for this model
NAME = "Cats-vs-dog-cnn-64x2-{}".format(int(time.time()))

# Specify the callback object
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(NAME))

# If u want to run several models at the same time, u can specify a fraction of GPU u want that model to take
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Build the model structure
model = Sequential()

model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten()) # Dense layer is 1D, Conv2D is 2D so we need to flatten it
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation("sigmoid"))

# In this case there are only 2 lables so we can use binary loss
model.compile(loss = "binary_crossentropy",
              optimizer = "adam",
              metrics = ['accuracy'])


"3. Train the model"
# Run the fitment, batch_size can be around 20-200 (not 1 or all) depending on the data size
# validation_split is kind of out-of-sample data
# Pass the callback object/list to the fitment
model.fit(X, y , batch_size = 32, epochs = 10,  validation_split = 0.1, callbacks = [tensorboard])

# With tensorflow , u define the model before u load in, because model is normally just weights
# With keras, u can just load in the model without defining the model
model.save('64x3-CNN.model')